# Preprocessing

## Constants

In [1]:
# Percentage of posts that will be pulled from each author's dataset as the testing set.
PERCENTAGE_FOR_TESTING = 0.2

# Minimum number of posts
MIN_NUM_POSTS = 5

# Minimum number of words
MIN_NUM_WORDS = 1000

# Determines whether or not a list of posts attributed to a single author
# will be shuffled before being separated into training and testing datasets
WILL_SHUFFLE = True

## Globals

In [2]:
testing_data = []
training_data = []
all_word_frequency = {}

# List of data that did not make the minimum amount of data threshold
leftover_data = []

## Read in Posts

This defines a function that takes in the filename of an author's 
dataset file, reads the data, and returns the id number of the author
and a list of the author's posts. 

In [3]:
import re

def get_posts(data_directory: str, data_filename: str):
    # Get the author id using regex
    a_id = re.search("^(\d+)", data_filename).group(1)
    
    # Get the array of the author's posts
    post_list = []
    file = open(data_directory + data_filename, "r", encoding='latin1')
    post_mode = False
    post = ""
    for line in file.readlines():
        line = line.strip()
        if re.search("^<post>$", line):
            # Found the start of a post
            post_mode = True
        elif re.search("^</post>$", line):
            # End of the post. Clean up the post and add it to the list
            post_mode = False
            post = post.strip()
            post_list.append(post)
            post = ""
        elif post_mode:
            # Concatenate the line to the post
            post += line
    
    return a_id, post_list

## Count the Data

Counts the number of words within a list of posts attributed to a single author.
It returns the total number of words within the whole list and the number of posts.
It will also add the words to the dictionary containing the frequency of words within the entire
corpus.

In [4]:
import spacy

def count_data(author_posts: list):
    global all_word_frequency
    
    nlp = spacy.load("en_core_web_sm")
    
    # Counters
    num_words = 0
    num_posts = len(author_posts)
    
    for post in author_posts:
        doc = nlp(post)
        
        for token in doc:
            word = token.text
            
            # Add to the total word count of this series of posts
            num_words += 1
            
            # Add the word to the global word frequency of the whole corpus
            if word in all_word_frequency.keys():
                all_word_frequency[word] += 1
            else:
                all_word_frequency[word] = 1
    
    return num_words, num_posts

## Add the Data to Dataset

Adds the post list to the specified global dataset list with the 
provided author id as the tag. It assumes that all posts within the list
is attributed to the provided author id. The global dataset list should be one of the 
following:
- testing_data
- training_data
- leftover_data

In [5]:
def add_data_to_set(a_id: int, post_list: list, global_dataset: list):
    for post in post_list:
        post_info = {"author_id": a_id, "post": post}
        global_dataset.append(post_info)

## Separate Data

Separates the list of posts into the training dataset and the testing dataset
by the percentage outlined by PERCENTAGE_FOR_TESTING. It assumes that 
all of the posts within the list is attributed to the author id provided.
In addition, it assumes that there are enough posts to meet the MIN_NUM_POSTS
requirements, so it won't have enough posts to provide at least one post for
either the training or testing set.

In [6]:
import random
import math

def separate_data(a_id: int, post_list: list):
    global WILL_SHUFFLE
    global PERCENTAGE_FOR_TESTING
    
    if WILL_SHUFFLE:
        random.shuffle(post_list)
        
    # Separate the post list into two parts, where one is PERCENTAGE_FOR_TESTING
    # of the whole post list
    num_posts_testing = math.floor(len(post_list) * PERCENTAGE_FOR_TESTING)
    testing_set = post_list[:num_posts_testing]
    training_set = post_list[num_posts_testing:]
    
    # Now add them to the respective global dataset
    global testing_data
    global training_data
    add_data_to_set(a_id, testing_set, testing_data)
    add_data_to_set(a_id, training_set, training_data)

## Process Data and Output to Files

In [7]:
import os
directory = "blogs/"

# Go through all of the files
print("Reading through data...")
for filename in os.listdir(directory):
    if filename.endswith(".xml"):
        author_id, posts = get_posts(directory, filename)
        word_count, post_count = count_data(posts)
        
        # Determine if this data makes the cut
        if word_count >= MIN_NUM_WORDS and post_count >= MIN_NUM_POSTS:
            # Separate out the data into training and dataset
            separate_data(author_id, posts)
        else:
            # Add the data to the leftover data
            add_data_to_set(author_id, posts, leftover_data)
        print("Finished processing author", author_id)
    else:
        continue

# Now output the data into JSON files
import json

print("Outputting the data to json files...")
with open('training_set.json', 'w') as json_file:
    dataset = {"data": training_data}
    json.dump(dataset, json_file, indent = 2, sort_keys=True)
with open('testing_set.json', 'w') as json_file:
    dataset = {"data": testing_data}
    json.dump(dataset, json_file, indent = 2, sort_keys=True)
with open('leftover_set.json', 'w') as json_file:
    dataset = {"data": leftover_data}
    json.dump(dataset, json_file, indent = 4, sort_keys=True)
with open('vocabulary.json', 'w') as json_file:
    json.dump(all_word_frequency, json_file)
print("Processing completed...")    

Reading through data...
Finished processing author 4162441
Finished processing author 3489929
Finished processing author 3954575
Finished processing author 3364931
Finished processing author 3162067
Finished processing author 813360
Finished processing author 4028373
Finished processing author 3630901
Finished processing author 2467122
Finished processing author 3732850
Finished processing author 3846432
Finished processing author 3600967
Finished processing author 3753301
Finished processing author 4157968
Finished processing author 3699514
Finished processing author 2727849
Finished processing author 3791552
Finished processing author 4278694
Finished processing author 1618178
Finished processing author 669719
Finished processing author 3865169
Finished processing author 4310425
Finished processing author 4186320
Finished processing author 2221350
Finished processing author 2318045
Finished processing author 827534
Finished processing author 3842783
Finished processing author 3315654

Finished processing author 3306403
Finished processing author 3777485
Finished processing author 3869230
Finished processing author 1863460
Finished processing author 944569
Finished processing author 3211137
Finished processing author 4246925
Finished processing author 3391038
Finished processing author 3607859
Finished processing author 3923707
Finished processing author 1884464
Finished processing author 3492118
Finished processing author 3743035
Finished processing author 3429409
Finished processing author 3082333
Finished processing author 2724350
Finished processing author 4182601
Finished processing author 3851330
Finished processing author 3406811
Finished processing author 3821176
Finished processing author 4108125
Finished processing author 3334411
Finished processing author 4310608
Finished processing author 3422352
Finished processing author 2844168
Finished processing author 3191071
Finished processing author 3360218
Finished processing author 3575606
Finished processing a

Finished processing author 2146771
Finished processing author 986006
Finished processing author 4059818
Finished processing author 3057616
Finished processing author 4309186
Finished processing author 4277181
Finished processing author 3395214
Finished processing author 3394208
Finished processing author 3013483
Finished processing author 3596634
Finished processing author 4035105
Finished processing author 3704194
Finished processing author 3533430
Finished processing author 3523319
Finished processing author 3910390
Finished processing author 3822298
Finished processing author 4087223
Finished processing author 4257604
Finished processing author 2550368
Finished processing author 4046295
Finished processing author 3984458
Finished processing author 4231038
Finished processing author 3683113
Finished processing author 3512279
Finished processing author 4128525
Finished processing author 3762303
Finished processing author 3749013
Finished processing author 3825001
Finished processing a

Finished processing author 1634038
Finished processing author 3823888
Finished processing author 3714436
Finished processing author 3774367
Finished processing author 3786992
Finished processing author 4323608
Finished processing author 3881934
Finished processing author 3354517
Finished processing author 3830714
Finished processing author 61176
Finished processing author 2020552
Finished processing author 3556371
Finished processing author 3973262
Finished processing author 3896598
Finished processing author 3786302
Finished processing author 4165198
Finished processing author 2685566
Finished processing author 3844773
Finished processing author 3907370
Finished processing author 3625825
Finished processing author 3745479
Finished processing author 3181599
Finished processing author 3706114
Finished processing author 3764815
Finished processing author 3869305
Finished processing author 3622804
Finished processing author 3425653
Finished processing author 3956807
Finished processing au

Finished processing author 3872328
Finished processing author 3445814
Finished processing author 1583261
Finished processing author 3983737
Finished processing author 4193809
Finished processing author 3727078
Finished processing author 2645906
Finished processing author 3378674
Finished processing author 4045469
Finished processing author 3484215
Finished processing author 3311279
Finished processing author 2155463
Finished processing author 3987552
Finished processing author 3521268
Finished processing author 2983072
Finished processing author 3843212
Finished processing author 3418388
Finished processing author 3659230
Finished processing author 4244468
Finished processing author 3672856
Finished processing author 4135893
Finished processing author 3389924
Finished processing author 3658568
Finished processing author 3686021
Finished processing author 3993587
Finished processing author 3784883
Finished processing author 3543172
Finished processing author 3616331
Finished processing 

...

Finished processing author 3898668
Finished processing author 3317569
Finished processing author 4258547
Finished processing author 3768930
Finished processing author 3773538
Finished processing author 2972526
Finished processing author 4247939
Finished processing author 4156981
Finished processing author 3416689
Finished processing author 4153678
Finished processing author 3588446
Finished processing author 3023198
Finished processing author 3198682
Finished processing author 3733281
Finished processing author 3441237
Finished processing author 3519080
Finished processing author 4180506
Finished processing author 4049704
Finished processing author 4112382
Finished processing author 3581115
Finished processing author 3696949
Finished processing author 2781029
Finished processing author 4027083
Finished processing author 3865126
Finished processing author 3658401
Finished processing author 3909699
Finished processing author 2873751
Finished processing author 3421911
Finished processing 

Finished processing author 3598702
Finished processing author 4312427
Finished processing author 828248
Finished processing author 4321311
Finished processing author 4293212
Finished processing author 4013023
Finished processing author 2020102
Finished processing author 3318771
Finished processing author 3037713
Finished processing author 3092694
Finished processing author 4208386
Finished processing author 3664471
Finished processing author 3440704
Finished processing author 3641942
Finished processing author 3638035
Finished processing author 2496003
Finished processing author 1533422
Finished processing author 4168844
Finished processing author 3588627
Finished processing author 3714729
Finished processing author 3526907
Finished processing author 801554
Finished processing author 3865755
Finished processing author 4069224
Finished processing author 4055095
Finished processing author 4317294
Finished processing author 865087
Finished processing author 1283349
Finished processing aut

Finished processing author 123532
Finished processing author 3342044
Finished processing author 4035743
Finished processing author 3291491
Finished processing author 3768768
Finished processing author 4211445
Finished processing author 3930071
Finished processing author 4323647
Finished processing author 3868857
Finished processing author 4214472
Finished processing author 3953356
Finished processing author 1131329
Finished processing author 3334428
Finished processing author 3864205
Finished processing author 3580456
Finished processing author 2081563
Finished processing author 4248010
Finished processing author 2952952
Finished processing author 4152878
Finished processing author 3995041
Finished processing author 3378096
Finished processing author 3583050
Finished processing author 3875132
Finished processing author 4131346
Finished processing author 3356810
Finished processing author 3315652
Finished processing author 4067717
Finished processing author 4285535
Finished processing a

Finished processing author 2864451
Finished processing author 3859699
Finished processing author 3583112
Finished processing author 3305513
Finished processing author 2894986
Finished processing author 1544534
Finished processing author 3732960
Finished processing author 3987266
Finished processing author 4047605
Finished processing author 4049328
Finished processing author 4129961
Finished processing author 907450
Finished processing author 3749319
Finished processing author 4247269
Finished processing author 3873505
Finished processing author 3407822
Finished processing author 3838046
Finished processing author 3840100
Finished processing author 3996741
Finished processing author 4277394
Finished processing author 3522746
Finished processing author 3983558
Finished processing author 2653067
Finished processing author 3914258
Finished processing author 3339874
Finished processing author 3800453
Finished processing author 3091107
Finished processing author 2959815
Finished processing a

Finished processing author 3696755
Finished processing author 3679664
Finished processing author 3228371
Finished processing author 3646134
Finished processing author 3916402
Finished processing author 3757802
Finished processing author 3543749
Finished processing author 3166930
Finished processing author 3819196
Finished processing author 4274878
Finished processing author 4049799
Finished processing author 3961110
Finished processing author 681976
Finished processing author 1682363
Finished processing author 790651
Finished processing author 3735527
Finished processing author 1826722
Finished processing author 3956585
Finished processing author 3515751
Finished processing author 3466286
Finished processing author 1636139
Finished processing author 4292814
Finished processing author 4067405
Finished processing author 3337697
Finished processing author 3096879
Finished processing author 3816523
Finished processing author 3749593
Finished processing author 1813392
Finished processing au